In [208]:
from bs4 import BeautifulSoup
import requests
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from datetime import date
from statistics import mean, median
from time import sleep
from datetime import datetime, timedelta
from selenium.webdriver.common.action_chains import ActionChains
from tqdm import tqdm

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import scipy.stats as stats
from statsmodels.miscmodels.ordinal_model import OrderedModel

In [411]:
df = pd.DataFrame({'Артикул':[], 'Цена': [], 'Название компании':[], 'Название товара': [], 'Количество отзывов': [], 
                   'Цвет': [],  'Рейтинг продавца':[], 'Рейтинг товара': [], 'Купили': [], '5 звезд': [], '4 звезды': [], '3 звезды': []
                  , '2 звезды': [], '1 звезда': []})



In [412]:
url = 'https://www.wildberries.ru/catalog/141734116/detail.aspx'

In [413]:
driver = webdriver.Chrome(r'/Users/konstantin_sukhin/Desktop/WB/chromedriver')
driver.get(url)


<ipython-input-413-8eb49c28eea1>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'/Users/konstantin_sukhin/Desktop/WB/chromedriver')


In [372]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
urls = soup.find(attrs={"class":{"catalog-page__main new-size"}}).find_all(attrs={"class":{"product-card__link j-card-link j-open-full-product-card"}})
len(urls)

100

In [391]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
urls += soup.find(attrs={"class":{"catalog-page__main new-size"}}).find_all(attrs={"class":{"product-card__link j-card-link j-open-full-product-card"}})
len(urls)

2000

In [417]:
urls[461:]

[<a class="product-card__link j-card-link j-open-full-product-card" draggable="false" href="https://www.wildberries.ru/catalog/13646538/detail.aspx" target="_blank"></a>,
 <a class="product-card__link j-card-link j-open-full-product-card" draggable="false" href="https://www.wildberries.ru/catalog/40167044/detail.aspx" target="_blank"></a>,
 <a class="product-card__link j-card-link j-open-full-product-card" draggable="false" href="https://www.wildberries.ru/catalog/9763222/detail.aspx" target="_blank"></a>,
 <a class="product-card__link j-card-link j-open-full-product-card" draggable="false" href="https://www.wildberries.ru/catalog/12212235/detail.aspx" target="_blank"></a>,
 <a class="product-card__link j-card-link j-open-full-product-card" draggable="false" href="https://www.wildberries.ru/catalog/19118572/detail.aspx" target="_blank"></a>,
 <a class="product-card__link j-card-link j-open-full-product-card" draggable="false" href="https://www.wildberries.ru/catalog/131750855/detail.as

In [426]:
for url in tqdm(urls[1279:]):
    url = url.get('href')
    driver.get(url)
    sleep(2.5)
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight-1300);var lenOfPage=document.body.scrollHeight-1300;return lenOfPage;")
    match=False
    while(match==False):
        lastCount = lenOfPage
        sleep(1.3)
        lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight-1300);var lenOfPage=document.body.scrollHeight-1300;return lenOfPage;")
        if lastCount==lenOfPage:
            match=True
    sleep(1.5)
    try:
        soup=BeautifulSoup(driver.page_source, 'html.parser')
        if soup.find(attrs={"class":{"user-opinion__text"}}) != None:


            element = driver.find_element(By.CLASS_NAME,  'user-opinion__text' )


            action = ActionChains(driver)
            action.move_to_element(to_element=element).perform()
            scroll_vаlue = 10
            scroll_by = f'window.scrollBy(0, {scroll_vаlue});'
            driver.execute_script(scroll_by)  
            action.move_to_element(to_element=element).perform()
            sleep(1.5)
            soup=BeautifulSoup(driver.page_source, 'html.parser')
            article = re.findall(r'\d+', soup.find(attrs={"class":{"product-article"}}).text)[0]
            if soup.find(attrs={"class":{"price-block__final-price"}}) != None:
                coast = soup.find(attrs={"class":{"price-block__final-price"}}).text.strip().replace('\xa0', '').replace('₽', '')
            seller_name = soup.find(attrs={"class":{"seller-info__title"}}).text.strip()
            product_name = soup.find(attrs={"class":{"product-page__header"}}).span.text + ' / ' +soup.find(attrs={"class":{"product-page__header"}}).h1.text
            count_feedbacs = re.findall(r'\d+', soup.find(attrs={"class":{"product-review__count-review"}}).text.replace(' ', ''))[0]
            try:
                color = soup.find(attrs={"class":{"color"}}).text
            except:
                color = ''
            if soup.find(attrs={"class":{"address-rate-mini"}})!=None:
                rating_saller = soup.find(attrs={"class":{"address-rate-mini"}}).text
            else:
                rating_saller = 0
            rating_good = soup.find(attrs={"class":{"user-opinion__rating-numb"}}).text
            if soup.find(attrs={"class":{"product-order-quantity j-orders-count-wrapper"}}).text != '':
                count_cell = re.findall(r'\d+', soup.find(attrs={"class":{"product-order-quantity j-orders-count-wrapper"}}).text.replace('\xa0', ''))[0]
            else:
                count_cell = 0
            st5 = soup.find_all(attrs={"class":{"feedback-percent__count"}})[0].text
            st4 = soup.find_all(attrs={"class":{"feedback-percent__count"}})[1].text
            st3 = soup.find_all(attrs={"class":{"feedback-percent__count"}})[2].text
            st2 = soup.find_all(attrs={"class":{"feedback-percent__count"}})[3].text
            st1 = soup.find_all(attrs={"class":{"feedback-percent__count"}})[4].text
            df = df.append({'Артикул':article, 'Цена': coast, 'Название компании':seller_name, 'Название товара': product_name, 'Количество отзывов': count_feedbacs, 
                               'Цвет': color,  'Рейтинг продавца':rating_saller, 'Рейтинг товара': rating_good, 'Купили': count_cell, '5 звезд': st5, '4 звезды': st4, '3 звезды': st3
                              , '2 звезды': st2, '1 звезда': st1}, ignore_index=True)
        else:
            soup=BeautifulSoup(driver.page_source, 'html.parser')
            article = re.findall(r'\d+', soup.find(attrs={"class":{"product-article"}}).text)[0]
            if soup.find(attrs={"class":{"price-block__final-price"}}) != None:
                coast = soup.find(attrs={"class":{"price-block__final-price"}}).text.strip().replace('\xa0', '').replace('₽', '')
            seller_name = soup.find(attrs={"class":{"seller-info__title"}}).text.strip()
            product_name = soup.find(attrs={"class":{"product-page__header"}}).span.text + ' / ' +soup.find(attrs={"class":{"product-page__header"}}).h1.text
            count_feedbacs = re.findall(r'\d+', soup.find(attrs={"class":{"product-review__count-review"}}).text.replace(' ', ''))[0]
            try:
                color = soup.find(attrs={"class":{"color"}}).text
            except:
                color = ''
            if soup.find(attrs={"class":{"address-rate-mini"}})!= None:
                rating_saller = soup.find(attrs={"class":{"address-rate-mini"}}).text
            else:
                rating_saller = 0
            try:
                rating_good = soup.find(attrs={"class":{"user-opinion__rating-numb"}}).text
            except:
                rating_good = ''
            if soup.find(attrs={"class":{"product-order-quantity j-orders-count-wrapper"}}).text != '':
                count_cell = re.findall(r'\d+', soup.find(attrs={"class":{"product-order-quantity j-orders-count-wrapper"}}).text.replace('\xa0', ''))[0]
            else:
                count_cell = 0
            df = df.append({'Артикул':article, 'Цена': coast, 'Название компании':seller_name, 'Название товара': product_name, 'Количество отзывов': count_feedbacs, 
                               'Цвет': color,  'Рейтинг продавца':rating_saller, 'Рейтинг товара': rating_good, 'Купили': count_cell}, ignore_index=True)

    except:
        soup=BeautifulSoup(driver.page_source, 'html.parser')
        article = re.findall(r'\d+', soup.find(attrs={"class":{"product-article"}}).text)[0]
        if soup.find(attrs={"class":{"price-block__final-price"}}) != None:
            coast = soup.find(attrs={"class":{"price-block__final-price"}}).text.strip().replace('\xa0', '').replace('₽', '')
        seller_name = soup.find(attrs={"class":{"seller-info__title"}}).text.strip()
        product_name = soup.find(attrs={"class":{"product-page__header"}}).span.text + ' / ' +soup.find(attrs={"class":{"product-page__header"}}).h1.text
        count_feedbacs = re.findall(r'\d+', soup.find(attrs={"class":{"product-review__count-review"}}).text.replace(' ', ''))[0]
        try:
            color = soup.find(attrs={"class":{"color"}}).text
        except:
            color = ''
        if soup.find(attrs={"class":{"address-rate-mini"}})!=None:
            rating_saller = soup.find(attrs={"class":{"address-rate-mini"}}).text
        else:
            rating_saller = 0
        try:
            rating_good = soup.find(attrs={"class":{"user-opinion__rating-numb"}}).text
        except:
            rating_good = ''
        if soup.find(attrs={"class":{"product-order-quantity j-orders-count-wrapper"}}).text != '':
            count_cell = re.findall(r'\d+', soup.find(attrs={"class":{"product-order-quantity j-orders-count-wrapper"}}).text.replace('\xa0', ''))[0]
        else:
            count_cell = 0
        df = df.append({'Артикул':article, 'Цена': coast, 'Название компании':seller_name, 'Название товара': product_name, 'Количество отзывов': count_feedbacs, 
                           'Цвет': color,  'Рейтинг продавца':rating_saller, 'Рейтинг товара': rating_good, 'Купили': count_cell}, ignore_index=True)

100%|██████████| 721/721 [2:07:44<00:00, 10.63s/it]  


In [428]:
#df.to_excel("headphones4000.xlsx")

In [423]:
#df.to_excel("headphones3_4277.xlsx")

In [420]:
#df.to_excel("headphones3_3936.xlsx")

In [416]:
#df.to_excel("headphones3_3460.xlsx")

In [271]:
#df.to_excel("headphones2_3000.xlsx")

In [65]:
#df.to_excel("headphones2000.xlsx")

In [2]:
df2000_st = pd.read_excel('headphones2000.xlsx')
df4000_st = pd.read_excel('headphones4000.xlsx')

df4000 = df2000_st.append(df4000_st).reset_index(drop=True)

In [3]:
def numbers(text):
    text = str(text)
    text = text.replace("\xa0", "")
    return int(re.findall(r'\d+', text)[0])

In [4]:
nona = df4000.copy()

In [5]:
nona['Количество отзывов'] = nona['Количество отзывов'].astype(float)
nona['Рейтинг продавца'] = nona['Рейтинг продавца'].astype(float)
nona['Цена'] = nona['Цена'].apply(numbers)
nona.loc[(nona['5 звезд'].isna()) & (nona['Количество отзывов']==0),'5 звезд'] = 0
nona.loc[(nona['4 звезды'].isna()) & (nona['Количество отзывов']==0),'4 звезды'] = 0
nona.loc[(nona['3 звезды'].isna()) & (nona['Количество отзывов']==0),'3 звезды'] = 0
nona.loc[(nona['2 звезды'].isna()) & (nona['Количество отзывов']==0),'2 звезды'] = 0
nona.loc[(nona['1 звезда'].isna()) & (nona['Количество отзывов']==0),'1 звезда'] = 0
nona.loc[(nona['Рейтинг товара'].isna()) & (nona['Количество отзывов']==0),'Рейтинг товара'] = 0
nona['Цвет'] = nona['Цвет'].fillna('no_color')
nona = nona.dropna()
nona['5 звезд'] = nona['5 звезд'].apply(numbers)
nona['4 звезды'] = nona['4 звезды'].apply(numbers)
nona['3 звезды'] = nona['3 звезды'].apply(numbers)
nona['2 звезды'] = nona['2 звезды'].apply(numbers)
nona['1 звезда'] = nona['1 звезда'].apply(numbers)
nona['Купили'] = nona['Купили'].apply(numbers)
nona['Купили'] = nona['Купили'].astype(int)
nona.loc[(nona['Рейтинг товара']=='') & (nona['Количество отзывов']==0),'Рейтинг товара'] = 0
nona['Рейтинг товара'] = nona['Рейтинг товара'].astype(float)

In [361]:
nona_X = nona.copy().reset_index(drop=True)
nona_X['tt'] = nona_X['Купили']-nona_X['Количество отзывов']
nona_X.loc[nona_X['tt']<0,'tt'] = np.NaN
nona_X = nona_X.dropna()

nona_X.loc[nona_X['tt']<10,'target']=0
nona_X.loc[(nona_X['tt']>=10) & (nona_X['tt']<100),'target']=1
nona_X.loc[(nona_X['tt']>=100) & (nona_X['tt']<500),'target']=2
nona_X.loc[(nona_X['tt']>=500) & (nona_X['tt']<2000),'target']=3
nona_X.loc[(nona_X['tt']>=2000),'target']=4


nona_X['index'] = nona_X['Название компании'].astype(str) + '_' + nona_X['Название товара'].astype(str) + '_' + nona_X['Цена'].astype(str)
nona_X.drop_duplicates(subset='index',inplace=True)
nona_X = nona_X.reset_index(drop=True)

In [422]:
mean_df = nona_X.groupby(['target'])[['Купили','tt']].mean().reset_index()
mean_df

,target,Купили,tt
0,0.0,6.857143,3.473469
1,1.0,65.654135,42.311278
2,2.0,410.646388,262.581749
3,3.0,1390.410959,1029.534247
4,4.0,19276.500000,15369.635000


In [427]:
mean_df = nona_X.groupby(['target'])[['Купили','tt']].mean().reset_index()
dd = (nona_X['target'].value_counts()/nona_X['target'].value_counts().sum()).reset_index()
dd = dd.rename(columns={'index':'Категория','target':'Доля наблюдений в категории'})
dd['Диапазон продаж сверх отзывов'] = ['>= 0 & <10',
                                       '>=10 & <100',
                                       '>=100 & <500',
                                       '>=500 & <2000',
                                       '>=2000 & < inf']
dd = dd.merge(mean_df,left_on='Категория',right_on='target')
dd = dd.rename(columns={'Купили':'Все продажи в среднем','tt':'Продажи сверх отзывов в среднем'})
dd = dd[['Категория','Диапазон продаж сверх отзывов','Все продажи в среднем',
         'Продажи сверх отзывов в среднем','Доля наблюдений в категории']]
dd#.to_excel('logit_table.xlsx')

,Категория,Диапазон продаж сверх отзывов,Все продажи в среднем,Продажи сверх отзывов в среднем,Доля наблюдений в категории
0,0.0,>= 0 & <10,6.857143,3.473469,0.476283
1,1.0,>=10 & <100,65.654135,42.311278,0.258554
2,2.0,>=100 & <500,410.646388,262.581749,0.102255
3,3.0,>=500 & <2000,1390.410959,1029.534247,0.085148
4,4.0,>=2000 & < inf,19276.500000,15369.635000,0.077760


In [428]:
#nona_X['show'].value_counts()

In [429]:
headphones = nona_X.copy()
headphones['Цена^2'] = headphones['Цена']**2
headphones = headphones.drop(columns=['Название компании','Название товара',
                                      'Цвет','Рейтинг товара',
                                      'Купили','Артикул',
                                      '1 звезда',
                                      #'3 звезды',
                                      #'5 звезд',
                                      #'show',
                                      'Unnamed: 0',
                                     'index','tt'])
headphones = headphones.reset_index(drop=True)
headphones.head()

,Цена,Количество отзывов,Рейтинг продавца,5 звезд,4 звезды,3 звезды,2 звезды,target,Цена^2
0,527,32767.0,4.4,83,5,3,2,4.0,277729
1,700,27311.0,4.5,84,6,3,2,4.0,490000
2,1050,4688.0,4.8,93,3,1,1,4.0,1102500
3,587,6477.0,4.6,86,4,3,2,4.0,344569
4,990,32767.0,4.8,92,3,2,1,4.0,980100


In [433]:
#headphones[headphones.columns[:-1]].describe().T.to_excel('desc_reviews.xlsx')
desc = pd.read_excel('desc_reviews.xlsx')
desc = desc.rename(columns={'Unnamed: 0':'Признаки'})
desc['Среднее значение'] = desc['Среднее значение'].round(2)
desc['Стандартное отклонение'] = desc['Стандартное отклонение'].round(2)
desc.to_excel('table_4.xlsx')

In [431]:
X = headphones.drop(columns=['target'])
Y = headphones['target']

In [432]:
mod_prob = OrderedModel(Y,
                        X,distr='logit')

res_prob = mod_prob.fit(method='powell')
res_prob.summary()

Optimization terminated successfully.
         Current function value: 0.900887
         Iterations: 16
         Function evaluations: 2916


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                 target   Log-Likelihood:                -2317.1
Model:                   OrderedModel   AIC:                             4650.
Method:            Maximum Likelihood   BIC:                             4697.
Date:                Wed, 10 May 2023                                         
Time:                        11:27:47                                         
No. Observations:                2572                                         
Df Residuals:                    2560                                         
Df Model:                           8                                         
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Цена                  -0.0001   1.47e-05     -7.669      0.000      -0.000   -8.37e-05
Количество отзывов     0.0055      0.000     17.610      0.000       0.005       0.006
Рейтинг продавца       0.8488      0.132      6.423      0.000       0.590       1.108
5 звезд                0.0333      0.001     22.664      0.000       0.030       0.036
4 звезды               0.0237      0.004      6.713      0.000       0.017       0.031
3 звезды               0.0295      0.004      6.971      0.000       0.021       0.038
2 звезды               0.0156      0.004      4.321      0.000       0.009       0.023
Цена^2              3.945e-09   8.26e-08      0.048      0.962   -1.58e-07    1.66e-07
0.0/1.0                5.5525      0.565      9.826      0.000       4.445       6.660
1.0/2.0                0.6612      0.035     19.108      0.000       0.593       0.729
2.0/3.0                0.1741      0.058      2.988      0.003       0.060       0.288
3.0/4.0                0.7623      0.069     11.060      0.000       0.627       0.897
======================================================================================
"""

In [401]:
params = res_prob.params
params

Цена                 -1.123912e-04
Количество отзывов    5.481564e-03
Рейтинг продавца      8.487950e-01
5 звезд               3.326958e-02
4 звезды              2.367332e-02
3 звезды              2.948176e-02
2 звезды              1.556957e-02
Цена^2                3.945388e-09
0.0/1.0               5.552458e+00
1.0/2.0               6.612276e-01
2.0/3.0               1.740862e-01
3.0/4.0               7.622686e-01
dtype: float64